<a href="https://colab.research.google.com/github/gollum-98/Sentiment-analysis-LSTM-Word-Embeddings/blob/master/IMDB_reviews_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
import tensorflow_datasets as tfds

In [3]:
imdb, info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)
train_data, test_data = imdb["train"], imdb["test"]
training_sentences = []
training_labels = []
test_sentences= []
test_labels = []
for s,l in train_data:
  training_sentences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())
for s,l in test_data:
  test_sentences.append(s.numpy().decode("utf8"))
  test_labels.append(l.numpy())

In [4]:
print(test_sentences[1])

A blackly comic tale of a down-trodden priest, Nazarin showcases the economy that Luis Bunuel was able to achieve in being able to tell a deeply humanist fable with a minimum of fuss. As an output from his Mexican era of film making, it was an invaluable talent to possess, with little money and extremely tight schedules. Nazarin, however, surpasses many of Bunuel's previous Mexican films in terms of the acting (Francisco Rabal is excellent), narrative and theme.<br /><br />The theme, interestingly, is something that was explored again in Viridiana, made three years later in Spain. It concerns the individual's struggle for humanity and altruism amongst a society that rejects any notion of virtue. Father Nazarin, however, is portrayed more sympathetically than Sister Viridiana. Whereas the latter seems to choose charity because she wishes to atone for her (perceived) sins, Nazarin's whole existence and reason for being seems to be to help others, whether they (or we) like it or not. The 

In [5]:
import numpy as np
training_labels = np.array(training_labels)
test_labels = np.array(test_labels)

In [6]:
print(test_labels[:20])

[1 1 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0]


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
max_length = 150
vocab_size = 10000
emb_dim = 16
tokenizer = Tokenizer(num_words=vocab_size,oov_token="<UNK>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
training_seq = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_seq,maxlen=max_length,padding='post')

test_seq = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seq,maxlen = max_length,padding='post')

In [16]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Embedding(vocab_size,emb_dim,input_length=max_length),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.LSTM(64,return_sequences=True),
                                    tf.keras.layers.LSTM(64),
                                    tf.keras.layers.Dense(32,activation="relu"),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(16,activation="relu"),
                                    #tf.keras.layers.Dropout(0,3),
                                    tf.keras.layers.Dense(1,activation = "sigmoid")
])
model.compile(loss="binary_crossentropy",optimizer = "adam",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 16)           160000    
_________________________________________________________________
dropout (Dropout)            (None, 150, 16)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 150, 64)           20736     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                5

In [17]:
history = model.fit(training_padded,training_labels,epochs=10,validation_data=(test_padded,test_labels),verbose=1)

Epoch 1/10
782/782 [==============================] - 172s 220ms/step - loss: 0.6912 - accuracy: 0.5143 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 2/10
782/782 [==============================] - 192s 245ms/step - loss: 0.6936 - accuracy: 0.4972 - val_loss: 0.6924 - val_accuracy: 0.5004
Epoch 3/10
782/782 [==============================] - 190s 243ms/step - loss: 0.6786 - accuracy: 0.5636 - val_loss: 0.6787 - val_accuracy: 0.5596
Epoch 4/10
782/782 [==============================] - 185s 237ms/step - loss: 0.6188 - accuracy: 0.6439 - val_loss: 0.4845 - val_accuracy: 0.7771
Epoch 5/10
782/782 [==============================] - 189s 242ms/step - loss: 0.3733 - accuracy: 0.8418 - val_loss: 0.3353 - val_accuracy: 0.8575
Epoch 6/10
782/782 [==============================] - 187s 239ms/step - loss: 0.2622 - accuracy: 0.8972 - val_loss: 0.3211 - val_accuracy: 0.8659
Epoch 7/10
782/782 [==============================] - 186s 238ms/step - loss: 0.2193 - accuracy: 0.9151 - val_loss: 0.3349 -